In [1]:
import pandas as pd
from sqlalchemy import create_engine
import urllib
import logging

# Configuración del log
import logging
import urllib.parse
from sqlalchemy import create_engine
import pandas as pd

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_sqlalchemy_engine(server, database, driver):
    params = urllib.parse.quote_plus(
        f"DRIVER={{{driver}}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"Trusted_Connection=yes;"
    )
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
    return engine

def load_data(file_path, sep, date_cols=None, additional_transforms=None):
    df = pd.read_csv(file_path, sep=sep, on_bad_lines='skip')
    if date_cols:
        for col in date_cols:
            df[col] = pd.to_datetime(df[col]).dt.strftime('%Y-%m-%d')

   

    if additional_transforms:
        for col, value in additional_transforms.items():
            df[col] = value
            
    return df

def insert_to_sql(df, table_name, engine):
    try:
        num_rows = df.shape[0]  # Obtiene el número de filas del DataFrame
        df.to_sql(table_name, engine, if_exists='append', index=False)
        logger.info(f"Datos cargados exitosamente en {table_name} de SQL. Número de filas cargadas: {num_rows}")
    except Exception as e:
        logger.error(f"Ocurrió un error al cargar datos en {table_name}: {e}")

# Configuración de la conexión a SQL Server
server = 'SKYCONPRODB03'
database = 'Bonos'
driver = 'ODBC Driver 17 for SQL Server' 
engine = create_sqlalchemy_engine(server, database, driver)
        

# Información de los archivos y las tablas
files_and_tables = {
 "Cabecera_Payfile_Bono_Geografico": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Cabecera_Payfile_Bono_Geografico_Semana_del_1_al_5_de_5-2024.txt',',', ['FECHA_CREACION'],{'FUENTE': 'Cabecera_Payfile_Bono_Geografico', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
 "Detalle_Payfile_Bono_Geografico": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Detalle_Payfile_Bono_Geografico_Semana_del_1_al_5_de_5-2024.txt',',',None,{'FUENTE': 'Detalle_Payfile_Bono_Geografico', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
 "Detalle_Pago_Bonos_Geografico": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Detalle_Pago_Bonos_Geografico_Semana_del_1_al_5_de_5-2024.txt',';',['FECHA_INICIAL', 'FECHA_FINAL', 'FECHA_CARGA', 'FECHA_VALIDACION'],{'FUENTE': 'Detalle_Pago_Bonos_Geografico', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
 "Cabecera_Payfile_Bono_Domiciliado": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Cabecera_Payfile_Bono_Domiciliado_Semana_del_1_al_5_de_5-2024.txt',',',['FECHA_CREACION'],{'FUENTE': 'Cabecera_Payfile_Bono_Domiciliado', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
 "Detalle_Pago_Bonos_Domiciliado": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Detalle_Pago_Bonos_Domiciliado_Semana_del_1_al_5_de_5-2024.txt',';',['FECHA_INICIAL', 'FECHA_FINAL', 'FECHA_CARGA'],{'FUENTE': 'Detalle_Pago_Bonos_Domiciliado', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
 "Detalle_Payfile_Bono_Domiciliado": (r'C:\Users\JMHERNANDEZL\Corporacion Novavision , S. de R.L. de C.V\Lopez Gutierrez Fabiola Gabriela - Información&Control\Lemus\5.Bonos\script py\Archivos_para_cargar_Bonos\Detalle_Payfile_Bono_Domiciliado_Semana_del_1_al_5_de_5-2024.txt',',',None,{'FUENTE': 'Detalle_Payfile_Bono_Domiciliado', 'SEMANA': 'Semana del 29 al 05 de 05-2024'}),
}

# Procesamiento de los archivos
for table_name, (file_path, sep, date_cols, additional_transforms) in files_and_tables.items():
    df = load_data(file_path, sep, date_cols, additional_transforms)
    insert_to_sql(df, table_name, engine)

INFO:__main__:Datos cargados exitosamente en Cabecera_Payfile_Bono_Geografico de SQL. Número de filas cargadas: 35
INFO:__main__:Datos cargados exitosamente en Detalle_Payfile_Bono_Geografico de SQL. Número de filas cargadas: 54
INFO:__main__:Datos cargados exitosamente en Detalle_Pago_Bonos_Geografico de SQL. Número de filas cargadas: 460
INFO:__main__:Datos cargados exitosamente en Cabecera_Payfile_Bono_Domiciliado de SQL. Número de filas cargadas: 1
INFO:__main__:Datos cargados exitosamente en Detalle_Pago_Bonos_Domiciliado de SQL. Número de filas cargadas: 1
INFO:__main__:Datos cargados exitosamente en Detalle_Payfile_Bono_Domiciliado de SQL. Número de filas cargadas: 1
